In [1]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from joblib import load
import talib
import numpy as np
import pandas as pd

# Cantidad de dias a esperar para generar features
WAIT = 300  

# VaR calculation
def calculate_rolling_var(returns, window=15, confidence_level=0.95):
    # Calculate the VaR for each window
    rolling_var = returns.rolling(window=window).apply(lambda x: -np.percentile(x, (1 - confidence_level) * 100), raw=True)
    return rolling_var
    
def get_features(data): 
    # Input Model Features generation
   
    # Calculating RSI
    data['RSI'] = talib.RSI(data['Adj Close'], timeperiod=14)
    
    # Calculating MACD
    macd, macdsignal, macdhist = talib.MACD(data['Adj Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    data['MACD'] = macd
    data['MACD_Signal'] = macdsignal
    data['MACD_Hist'] = macdhist
    
    # Calculating ADX
    data['ADX'] = talib.ADX(data['High'], data['Low'], data['Adj Close'], timeperiod=14)
    
    # Calculating Bollinger Bands
    upperband, middleband, lowerband = talib.BBANDS(data['Adj Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    data['Boll_Upper'] = upperband
    data['Boll_Middle'] = middleband
    data['Boll_Lower'] = lowerband
    
    # Calculating OBV
    data['OBV'] = talib.OBV(data['Adj Close'], data['Volume'])
    
    # Calculating ATR
    data['ATR'] = talib.ATR(data['High'], data['Low'], data['Adj Close'], timeperiod=14)
    
    # Calculating Rolling Standard Deviation
    data['Rolling_STD'] = data['Adj Close'].rolling(window=252).std()
    
    # Calculating Daily Returns
    data['Daily_Returns'] = data['Adj Close'].pct_change()
    
    # Calculating Historical VaR (95% confidence)
    var_confidence_level = 0.95
    # Assuming 'Daily_Returns' are already calculated
    data['Rolling_VaR'] = calculate_rolling_var(data['Daily_Returns'])    

    #print(data.head())
    
    data = data.dropna()
    return data 
    
class MLStrategy(Strategy):
    def init(self):
        # init method provied by backtesting.py framework
        # it is not __init__ python operator 
        pass 
        
    def next(self):
        
        if len(self.data) <= WAIT:
            return 
  
        current_features = self.data.df[:-1]
        model_features = get_features(current_features)
        
        prediction = rf_classifier.predict(model_features[features])
        
        if prediction[-1] == 1:
            if not self.position:
                print(f"Buy {self.data.df.iloc[-1].name}")
                self.buy()
            
        else:
            if self.position:
                print("Close")
                self.position.close()
  
rf_classifier = load('rf_classifier_2.joblib')
# Read OHCLV csv file 
data = pd.read_csv("./SPLG.csv", parse_dates=True)
features =['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'RSI', 'MACD', 'MACD_Signal', 'MACD_Hist', 'ADX', 'Boll_Upper', 'Boll_Middle', 'Boll_Lower', 'OBV', 'ATR', 'Rolling_STD', 'Daily_Returns', 'Rolling_VaR', 'Adj Close_VIX']

# Read VIX csv file 
vix_file_path = './VIX.csv'
vix_data = pd.read_csv(vix_file_path)

# Cast to Datetime type 
data['Date'] = pd.to_datetime(data['Date'])
vix_data['Date'] = pd.to_datetime(vix_data['Date'])


# Merging the ETF data with VIX data
# Using an outer join to ensure all dates in either dataset are included
combined_data = pd.merge(data, vix_data[['Date', 'Adj Close']], on='Date', how='outer', suffixes=('', '_VIX'))

# Sorting by date to maintain chronological order
combined_data = combined_data.sort_values(by='Date')
combined_data.dropna(inplace=True)
combined_data.set_index('Date',inplace=True)

bt = Backtest(combined_data, MLStrategy, cash=10000, exclusive_orders=False,commission=.002) 
stats = bt.run()
print(stats)


/home/mamani/venv/FinalProject/lib/python3.8/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

Buy 2007-01-30 00:00:00
Close
Buy 2007-03-21 00:00:00
Close
Buy 2007-03-28 00:00:00
Close
Buy 2007-06-11 00:00:00
Close
Buy 2007-06-15 00:00:00
Close
Buy 2007-07-06 00:00:00
Close
Buy 2007-07-12 00:00:00
Close
Buy 2007-08-27 00:00:00
Close
Buy 2007-08-31 00:00:00
Close
Buy 2007-09-07 00:00:00
Close
Buy 2007-09-12 00:00:00
Close
Buy 2007-10-24 00:00:00
Close
Buy 2007-10-29 00:00:00
Close
Buy 2007-12-10 00:00:00
Close
Buy 2007-12-13 00:00:00
Close
Buy 2007-12-24 00:00:00
Close
Buy 2008-02-29 00:00:00
Close
Buy 2008-03-24 00:00:00
Close
Buy 2008-04-01 00:00:00
Close
Buy 2008-04-17 00:00:00
Close
Buy 2008-05-29 00:00:00
Close
Buy 2008-06-04 00:00:00
Close
Buy 2008-06-06 00:00:00
Close
Buy 2008-08-07 00:00:00
Close
Buy 2008-08-14 00:00:00
Close
Buy 2008-08-21 00:00:00
Close
Buy 2008-08-28 00:00:00
Close
Buy 2008-09-22 00:00:00
Close
Buy 2008-12-09 00:00:00
Close
Buy 2008-12-17 00:00:00
Close
Buy 2008-12-23 00:00:00
Close
Buy 2008-12-31 00:00:00
Close
Buy 2009-01-22 00:00:00
Close
Buy 2009-0

KeyboardInterrupt: 